In [1]:
pip install pyvi

     |████████████████████████████████| 8.5MB 11.0MB/s 
     |████████████████████████████████| 747kB 38.0MB/s 


In [2]:
from pyvi import ViTokenizer
import nltk
import numpy as np

import keras
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import LSTM

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers , activations , models , preprocessing , utils
import tensorflow as tf
from  sklearn.model_selection import train_test_split

from numpy import array
from numpy import argmax
from numpy import array_equal
from keras.utils import to_categorical
from keras.models import Model
from keras.layers import Input
from keras.layers import TimeDistributed

# Tập data chatbot

In [3]:
file_tranning = ['bạn bè.txt',
'các câu hỏi phức tạp.txt',
'đất nước.txt',
'địa chỉ.txt',
'du lịch.txt',
'gia đình.txt',
'giải trí.txt',
'học tập.txt',
'nghề nghiệp.txt',
'nghỉ lễ.txt',
'người yêu.txt',
'robot.txt',
'shoping.txt',
'tán gẫu.txt',
'tdtu.txt',
'thông tin cá nhân.txt',
'trò chuyện về đi ăn.txt']

In [4]:
question_train = []
answer_train = []

for i in range(len(file_tranning)):
    with open(file_tranning[i], encoding='UTF-8') as f:
        train_lines = f.readlines()
        for line in train_lines:
            tmp = line.split("__eou__")
            question_train.append(tmp[0].strip()) # strip(): Loại bỏ whitespace đầu cuối string
            answer_train.append(tmp[1].strip())

In [5]:
(question_train[:20])
# print((answer_train[0]))

['Thích đánh lộn không?',
 'Solo yasua không',
 'Mai đi picnic không?',
 'Mai học ca mấy vậy?',
 'Còn tiền không?',
 'Mấy giờ rồi nhỉ?',
 'Bao lâu rồi chưa về quê',
 'Bạn có crush chưa',
 'Crush bao lâu rồi',
 'Crush có Bồ chưa?',
 'Vậy sao còn crush ?',
 'Bạn có bạn trai chưa?',
 'Mai rảnh không rũ bồ đi bar chơi',
 'Bạn có thói quen làm gì khi rảnh vậy ?',
 'Khi bạn đi chơi cùng bạn bè thì nơi "tủ" của bạn là ở đâu vậy ?',
 'Bạn có thích hút thuốc lúc rảnh không ?',
 'Bạn có thích học bài lúc rảnh không ?',
 'Bạn có muốn đi bơi cùng bạn bè lúc rảnh không ?',
 'Dạo này bạn với crush sao rồi, ổn không ?',
 'Tỏ tình với crush chưa bạn, bữa tui thấy bạn đi chung với bạn nữ nào thì phải ?']

# Preprocessing

In [6]:
# Xóa question và answer khi answer empty string
list_index_empty_answer = []

In [7]:
for i in range(len(answer_train)):
    if(answer_train[i] == ""):
        list_index_empty_answer.append(i)

In [8]:
print(list_index_empty_answer)

[423, 424, 425, 426, 427, 428, 486, 487, 488, 489, 490, 632, 633, 648, 649, 650, 651, 652, 1173, 1174, 1175, 1176, 1177, 1178, 1179, 1180, 1229, 1230, 1231, 1232, 1233, 1814, 1815, 1816, 1817, 1871, 1872, 1873, 1874, 2372, 2373, 2374, 2375, 2376, 2377, 2653, 2654, 2703, 2704, 2705, 2706, 3240, 3241, 3242, 3243, 3244, 3289, 3290, 3291, 3292, 3293, 4195, 4196, 4197, 4198, 4199, 5030, 5031, 5032, 5033, 5034, 5035, 5405]


In [9]:
for i in range(len(list_index_empty_answer)):
    del answer_train[list_index_empty_answer[i]-i]
    del question_train[list_index_empty_answer[i]-i]

In [10]:
import string

In [11]:
def text_process(mess):
    # chuyển về chữ thường
    mess = mess.lower()
    
    # xóa dấu câu
    mess = [char for char in mess if char not in string.punctuation]
    mess = ''.join(mess)
    
    # replace whitespace
    mess = mess.replace("   ", " ")
    mess = mess.replace("  ", " ")
    
    # Word Segmentation
    mess = ViTokenizer.tokenize(mess)
    
    return mess

In [12]:
for i in range(len(question_train)):
    question_train[i] = text_process(question_train[i])
    answer_train[i] = text_process(answer_train[i])

In [13]:
#print((question_train[631]))
#print((answer_train[631]))

In [14]:
print(question_train[0])
print(answer_train[0])

thích đánh_lộn không
ngon nhà_vô


In [15]:
data = question_train
data_answer = answer_train

In [16]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(lower=True,filters='!"#$%&()*+,-./:;<=>?@[\]^`{|}~ ',split=' ')

tokenizer.fit_on_texts(data)
print(data[0])
print(data[1])
print(data[2])

tokenizer.fit_on_texts(data_answer)
print(data_answer[0])
print(data_answer[1])
print(data_answer[2])

thích đánh_lộn không
solo yasua không
mai đi picnic không
ngon nhà_vô
chấp lun 2 mạng đầu
mai bận học rồi


In [17]:
word2count = {}

for line in data:
    for word in line.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1
for line in data_answer:
    for word in line.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1

In [18]:
len(word2count)

4857

In [19]:
thresh = 1

vocab = {}
word_num = 0
for word, count in word2count.items():
    if count >= thresh:
        vocab[word] = word_num
        word_num += 1

In [20]:
len(vocab)

4857

In [21]:
for i in range(len(data_answer)):
    data_answer[i] = '<SOS> ' + data_answer[i] + ' <EOS>'


In [22]:
len(data_answer)

5537

In [23]:
tokens = ['<PAD>', '<EOS>', '<OUT>', '<SOS>']
x = len(vocab)
for token in tokens:
    vocab[token] = x
    x += 1

In [24]:
len(vocab)

4861

In [25]:
vocab['cameron'] = vocab['<PAD>']
vocab['<PAD>'] = 0

In [26]:
inv_vocab = {w:v for v, w in vocab.items()}
len(inv_vocab)

4861

In [27]:
encoder_inp = []
for line in data:
    lst = []
    for word in line.split():
        if word not in vocab:
            lst.append(vocab['<OUT>'])
        else:
            lst.append(vocab[word])
        
    encoder_inp.append(lst)

In [28]:
len(encoder_inp)

5537

In [29]:
decoder_inp = []
for line in data_answer:
    lst = []
    for word in line.split():
        if word not in vocab:
            lst.append(vocab['<OUT>'])
        else:
            lst.append(vocab[word])        
    decoder_inp.append(lst)

In [30]:
len(decoder_inp)

5537

In [31]:
encoder_inp = pad_sequences(encoder_inp, 15, padding='post', truncating='post')
decoder_inp = pad_sequences(decoder_inp, 15, padding='post', truncating='post')


In [32]:
decoder_final_output = []
for i in decoder_inp:
    decoder_final_output.append(i[1:]) 


In [33]:
decoder_final_output[:2]

[array([ 894, 2558, 4858,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0], dtype=int32),
 array([2559, 2514,  454,  492,  475, 4858,    0,    0,    0,    0,    0,
           0,    0,    0], dtype=int32)]

In [34]:
decoder_final_output = pad_sequences(decoder_final_output, 15, padding='post', truncating='post')
decoder_final_output[:2]

array([[ 894, 2558, 4858,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0],
       [2559, 2514,  454,  492,  475, 4858,    0,    0,    0,    0,    0,
           0,    0,    0,    0]], dtype=int32)

In [35]:
decoder_final_output = to_categorical(decoder_final_output, len(vocab))

In [36]:
print(decoder_final_output.shape)

(5537, 15, 4862)


In [37]:
enc_inp = Input(shape=(15, ))
dec_inp = Input(shape=(15, ))

In [38]:
VOCAB_SIZE = len(vocab)
embed = Embedding(VOCAB_SIZE+1, output_dim=64, 
                  input_length=15,
                  trainable=True                  
                  )

In [39]:
enc_embed = embed(enc_inp)
enc_lstm = LSTM(128, return_sequences=True, return_state=True)
enc_op, h, c = enc_lstm(enc_embed)
enc_states = [h, c]

In [42]:
dec_embed = embed(dec_inp)
dec_lstm = LSTM(128, return_sequences=True, return_state=True)
dec_op, _, _ = dec_lstm(dec_embed, initial_state=enc_states)

dense = Dense(VOCAB_SIZE, activation='softmax')

dense_op = dense(dec_op)

model = Model([enc_inp, dec_inp], dense_op)

model.compile(loss='categorical_crossentropy',metrics=['acc'],optimizer='adam')

model.fit([encoder_inp, decoder_inp],decoder_final_output,epochs=150,batch_size=32)

Epoch 1/150
174/174 [==============================] - 20s 96ms/step - loss: 5.0420 - acc: 0.4919
Epoch 2/150
174/174 [==============================] - 17s 97ms/step - loss: 2.8554 - acc: 0.5735
Epoch 3/150
174/174 [==============================] - 17s 97ms/step - loss: 2.6981 - acc: 0.5828
Epoch 4/150
174/174 [==============================] - 17s 97ms/step - loss: 2.5509 - acc: 0.5970
Epoch 5/150
174/174 [==============================] - 17s 101ms/step - loss: 2.4107 - acc: 0.6063
Epoch 6/150
174/174 [==============================] - 18s 101ms/step - loss: 2.2552 - acc: 0.6223
Epoch 7/150
174/174 [==============================] - 17s 98ms/step - loss: 2.2046 - acc: 0.6232
Epoch 8/150
174/174 [==============================] - 17s 96ms/step - loss: 2.1286 - acc: 0.6283
Epoch 9/150
174/174 [==============================] - 17s 97ms/step - loss: 2.0434 - acc: 0.6366
Epoch 10/150
174/174 [==============================] - 17s 96ms/step - loss: 1.9141 - acc: 0.6514
Epoch 11/150
174/

In [43]:
enc_model = Model([enc_inp], enc_states)

decoder_state_input_h = Input(shape=(128,))
decoder_state_input_c = Input(shape=(128,))

decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = dec_lstm(dec_embed , 
                                    initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]

dec_model = Model([dec_inp]+ decoder_states_inputs,
                                      [decoder_outputs]+ decoder_states)

In [44]:
print("##########################################")
print("#       start chatting        #")
print("##########################################")

prepro1 = ""
while prepro1 != 'q':
  prepro1  = input("you : ")
  prepro1 = text_process(prepro1)
  prepro = [prepro1]
  txt = []
  for x in prepro:
    lst = []
    for y in x.split():
      try:
        lst.append(vocab[y])
      except:
        lst.append(vocab['<OUT>'])
    txt.append(lst)
    txt = pad_sequences(txt, 15, padding='post')

    stat = enc_model.predict( txt )
    empty_target_seq = np.zeros( ( 1 , 1) )
    empty_target_seq[0, 0] = vocab['<SOS>']
    stop_condition = False

    decoded_translation = ''

    while not stop_condition :
      dec_outputs , h, c= dec_model.predict([empty_target_seq] + stat )
      decoder_concat_input = dense(dec_outputs)
      sampled_word_index = np.argmax( decoder_concat_input[0, -1, :] )
      sampled_word = inv_vocab[sampled_word_index] + ' '

      if sampled_word != '<EOS> ':
        decoded_translation += sampled_word
      if sampled_word == '<EOS> ' or len(decoded_translation.split()) > 13:
        stop_condition = True

      empty_target_seq = np.zeros( ( 1 , 1 ) )  
      empty_target_seq[ 0 , 0 ] = sampled_word_index

      stat = [h, c]
    print("chatbot attention : ", decoded_translation )
    print("==============================================")  

##########################################
#       start chatting        #
##########################################
you : xin chào
chatbot attention :  chào bạn bạn muốn tôi giúp gì 
you : bạn có phải robot
chatbot attention :  bạn có 20k 
you : bạn cho mình hả
chatbot attention :  bạn ấy bạn nhé 
you : bạn đang làm gì vậy
chatbot attention :  có 
you : bạn học trường nào
chatbot attention :  mình học trường tôn đức thắng 
you : bạn học khoa gì
chatbot attention :  mình học khoa công_nghệ thông_tin 
you : bạn có người yêu chưa
chatbot attention :  mình chưa có người_yêu 
you : giữa nam và nữ bạn chọn ai
chatbot attention :  quá xinh_đẹp là đằng khác 
you : vậy bạn ăn tối hay ăn trưa
chatbot attention :  lát mình ăn lẩu mẹ nấu 
you : oh. thế bạn có cho mình ăn cùng không
chatbot attention :  đương_nhiên rồi mình cũng <PAD> nấu_ăn bạn lắm 
you : bạn đi đâu đấy
chatbot attention :  đi về nhà 
you : nhà bạn ở đâu
chatbot attention :  nhà mình ở quận 11 
you : ở quận 11  có gì hay không
c

KeyboardInterrupt: ignored